In [1]:
import numpy as np
import heapq

In [2]:

class Graph:
  def __init__(self, numVertices):
    self.numVertices = numVertices
    self.adjMatrix = np.full((numVertices, numVertices), np.inf)
    self.minPaths = np.full((numVertices, 2), np.inf)

  def addEdge(self, src, dest, weight):
    self.adjMatrix[src][dest] = weight
    self.adjMatrix[dest][src] = weight

  def getWeight(self, src, dest):
    return self.adjMatrix[src][dest]
  
  def getNumVertices(self):
    return self.numVertices
  
  def printGraph(self):
    print(self.adjMatrix)

  def getNeighbors(self, vertex):
    indices = np.where(self.adjMatrix[vertex] != np.inf)
    neighbors = [(i, self.adjMatrix[vertex][i]) for i in indices[0]]
    return neighbors
  
  def computeMinPaths(self):
    for i in range(self.numVertices):
      sortedIndices = np.argsort(self.adjMatrix[i])
      self.minPaths[i] = (self.adjMatrix[i][sortedIndices[0]], self.adjMatrix[i][sortedIndices[1]])

class Node:
  def __init__(self, graph, prevNode=None, finalVertice=None):
    if prevNode is None or finalVertice is None:
      self.level = 1
      self.path = [0]
      self.cost = 0
      self.bound = bound(graph, 0)
      return

    self.level = prevNode.level + 1
    self.path = np.concatenate([prevNode.path, [finalVertice]]).astype(int)
    if len(prevNode.path) == 0:
      self.cost = 0
    else:
      self.cost = prevNode.cost + graph.getWeight(prevNode.path[-1], finalVertice)
    self.bound = bound(graph, finalVertice, prevNode)

  def __lt__(self, other):
    return self.bound < other.bound
  
  def __gt__(self, other):
    return self.bound > other.bound
  
  def __eq__(self, other):
    return self.bound == other.bound
  
  def __str__(self) -> str:
    return f"Path: {self.path} Cost: {self.cost} Bound: {self.bound}"


def bound(graph, finalVertice, prevNode=None):
  if prevNode is None:
    return np.sum(graph.minPaths) / 2
  
  cost = graph.getWeight(prevNode.path[-1], finalVertice)

  bound = prevNode.bound * 2

  if cost >= graph.minPaths[finalVertice][1]:
    bound -= graph.minPaths[finalVertice][1]
    bound += cost
  
  if cost >= graph.minPaths[prevNode.path[-1]][1]:
    bound -= graph.minPaths[prevNode.path[-1]][1]
    bound += cost

  return bound / 2

# def bound(graph, path, cost, finalVertice, unvisitedVertices=None):
#   unvisitedContributions = np.sum(graph.minPaths[np.isin(np.arange(graph.getNumVertices()), path) == False, :], axis=1)
#   if finalVertice != -1:
#     unvisitedContributions += graph.minPaths[finalVertice][0]
#   return (cost + np.sum(unvisitedContributions)) / 2


def branchBoundTravelingSalesman(graph):
  graph.computeMinPaths()

  root = Node(graph, None, 0)
  queue = [root]
  heapq.heapify(queue)
  best = np.inf
  solution = []

  numVertices = graph.getNumVertices()

  while queue:
    node = heapq.heappop(queue)
    
    if node.level >= numVertices:
      finalNode = Node(graph, node, 0)
      if finalNode.cost < best:
        best = finalNode.cost
        solution = finalNode.path
    
    elif node.bound < best:
      if node.level <= numVertices:
        unvisited = np.setdiff1d(np.arange(numVertices), node.path)
        for k in unvisited:
          newNode = Node(graph, node, k)
          if k not in node.path and graph.getWeight(node.path[-1], k) != np.inf and newNode.bound < best:
            heapq.heappush(queue, Node(graph, node, k))

  return solution, best

In [3]:
g = Graph(5)
g.addEdge(0, 1, 3)
g.addEdge(0, 2, 1)
g.addEdge(0, 3, 5)
g.addEdge(0, 4, 8)
g.addEdge(1, 2, 6)
g.addEdge(1, 3, 7)
g.addEdge(1, 4, 9)
g.addEdge(2, 3, 4)
g.addEdge(2, 4, 2)
g.addEdge(3, 4, 3)

# g.printGraph()
# print(g.getNeighbors(0))
# print(g.getNeighbors(1))
# print(g.getNeighbors(2))
# print(g.getNeighbors(3))
# print(g.getNeighbors(4))

g.computeMinPaths()
print(g.minPaths)

print('solution', branchBoundTravelingSalesman(g))


[[1. 3.]
 [3. 6.]
 [1. 2.]
 [3. 4.]
 [2. 3.]]
solution (array([0, 2, 4, 3, 1, 0]), 16.0)


In [4]:
g1 = Graph(4)
g1.adjMatrix = np.array([[0, 10, 15, 20],
       [10, 0, 35, 25],
       [15, 35, 0, 30],
       [20, 25, 30, 0]])

g1.printGraph()
g1.computeMinPaths()
print(g1.minPaths)

print('solution', branchBoundTravelingSalesman(g1))

[[ 0 10 15 20]
 [10  0 35 25]
 [15 35  0 30]
 [20 25 30  0]]
[[ 0. 10.]
 [ 0. 10.]
 [ 0. 15.]
 [ 0. 20.]]
solution (array([0, 1, 3, 2, 0]), 80)


In [5]:
class Point:
  def __init__(self, id, x, y):
    self.id = id
    self.x = x
    self.y = y

  def distance(self, point):
    return np.sqrt((self.x - point.x)**2 + (self.y - point.y)**2)
  
  def __str__(self) -> str:
    return f'Point {self.id} ({self.x}, {self.y})'

In [6]:
# function to read a file and return a list of lists
def readFile(file_name):
    with open(file_name) as f:
        content = f.readlines()
    content = [x.strip() for x in content if x[0].isdigit()]
    return content


In [7]:
# each element of the content list is a string with the id, x and y coordinates
# we need to split each line and create a Point object

content = readFile('berlin52.tsp')

points = []
for line in content:
  id, x, y = line.split(' ')
  points.append(Point(int(id)-1, float(x), float(y)))

for p in points:
  print(p)

Point 0 (565.0, 575.0)
Point 1 (25.0, 185.0)
Point 2 (345.0, 750.0)
Point 3 (945.0, 685.0)
Point 4 (845.0, 655.0)
Point 5 (880.0, 660.0)
Point 6 (25.0, 230.0)
Point 7 (525.0, 1000.0)
Point 8 (580.0, 1175.0)
Point 9 (650.0, 1130.0)
Point 10 (1605.0, 620.0)
Point 11 (1220.0, 580.0)
Point 12 (1465.0, 200.0)
Point 13 (1530.0, 5.0)
Point 14 (845.0, 680.0)
Point 15 (725.0, 370.0)
Point 16 (145.0, 665.0)
Point 17 (415.0, 635.0)
Point 18 (510.0, 875.0)
Point 19 (560.0, 365.0)


In [8]:
graph = Graph(len(points))

for p in points:
  for q in points:
    if p.id != q.id:
      graph.addEdge(p.id, q.id, p.distance(q))


In [9]:
graph.computeMinPaths()
print(graph.minPaths)

for line in graph.adjMatrix:
  print(line)

[[161.55494421 210.05951538]
 [ 45.         494.77267507]
 [134.62912018 207.00241544]
 [ 69.64194139 100.12492197]
 [ 25.          35.35533906]
 [ 35.35533906  40.31128874]
 [ 45.         451.24826869]
 [125.89678312 180.34688797]
 [ 83.21658489 183.43936328]
 [ 83.21658489 180.34688797]
 [387.07234466 442.71887242]
 [294.36372059 349.28498393]
 [205.54804791 442.71887242]
 [205.54804791 619.55629284]
 [ 25.          40.31128874]
 [165.07574019 260.04807248]
 [217.31313812 271.66155414]
 [134.62912018 161.55494421]
 [125.89678312 207.00241544]
 [165.07574019 210.05951538]]
[          inf  666.10809934  281.11385594  395.6008089   291.20439557
  326.26676202  640.8002809   426.87820277  600.18747071  561.47128155
 1040.97310244  655.01908369  975.         1120.76982472  299.04013109
  260.04807248  429.5346319   161.55494421  305.          210.05951538]
[ 666.10809934           inf  649.32657423 1047.09120902  945.14549145
  978.08486339   45.          956.15113868 1134.95594628 1132.9

In [10]:
# print('solution', branchBoundTravelingSalesman(graph))

KeyboardInterrupt: 